In [3]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from sqlalchemy import create_engine

# MySQL Database Configuration
db_config = {
    'user': 'root',
    'password': 'TM_edu',
    'host': 'localhost',
    'database': 'list_of_schools',
    'port': '3306'
}

# Create a MySQL connection
engine = create_engine(f"mysql+mysqlconnector://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['database']}")

# Query to fetch data from MySQL
query = 'SELECT * FROM your_table_name;'
df = pd.read_sql(query, con=engine)

# Create Autocomplete Widget
school_code_autocomplete = widgets.Text(
    value='',
    placeholder='Enter school code',
    description='School Code:',
    disabled=False
)

# Autocomplete Suggestions
initial_suggestion = 'Select'
school_code_autocomplete_suggestions = widgets.Dropdown(
    options=[initial_suggestion],
    value=initial_suggestion,
    description='Suggestions:',
    disabled=False,
)

# Create Search Button
search_button = widgets.Button(
    description='Search',
    disabled=False,
    button_style='',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to search',
    icon='search'  # (FontAwesome names without the `fa-` prefix)
)

# Create Clear Button
clear_button = widgets.Button(
    description='Clear',
    disabled=False,
    button_style='warning',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to clear results',
    icon='times-circle'  # (FontAwesome names without the `fa-` prefix)
)

# Display Output Result
output_result = widgets.Output()

# Define Search Function
def search_school(b):
    school_code = school_code_autocomplete.value
    result = df[df['KOD SEKOLAH'] == school_code][['KOD SEKOLAH', 'SENARAI SEKOLAH MALAYSIA', 'SEKOLAH INTERIM', 'SEKOLAH VSAT', 'SEKOLAH HIBRID']]
    
    with output_result:
        output_result.clear_output()
        if result.empty:
            print(f"No information found for School Code: {school_code}")
        else:
            # Display the output as a list
            print("Information for School Code {}: ".format(school_code))
            for _, row in result.iterrows():
                print("- School Code: {}".format(row['KOD SEKOLAH']))
                print("  School Name: {}".format(row['SENARAI SEKOLAH MALAYSIA']))
                print("  TM Interim: {}".format(row['SEKOLAH INTERIM']))
                print("  VSAT: {}".format(row['SEKOLAH VSAT']))
                print("  TM Hybrid: {}".format(row['SEKOLAH HIBRID']))
                print("\n")

# Define Clear Function
def clear_results(b):
    with output_result:
        output_result.clear_output()
    school_code_autocomplete.value = ''
    school_code_autocomplete_suggestions.options = [initial_suggestion]
    school_code_autocomplete_suggestions.value = initial_suggestion

# Function to update suggestions based on input
def update_autocomplete_suggestions(change):
    entered_text = change.new.upper()
    filtered_options = [option for option in df['KOD SEKOLAH'].tolist() if option.startswith(entered_text)]
    school_code_autocomplete_suggestions.options = [initial_suggestion] + filtered_options
    
# Function to update search bar based on dropdown selection
def update_search_bar(change):
    selected_option = change.new
    if selected_option != 'Select':
        school_code_autocomplete.value = selected_option
        
# Link the buttons to their respective functions
search_button.on_click(search_school)
clear_button.on_click(clear_results)

# Link the autocomplete widget to update suggestions
school_code_autocomplete.observe(update_autocomplete_suggestions, names='value')

# Link the dropdown to update search bar on selection
school_code_autocomplete_suggestions.observe(update_search_bar, names='value')

# Display widgets
display(school_code_autocomplete, school_code_autocomplete_suggestions, search_button, clear_button)

# Display the result output
display(output_result)


Text(value='', description='School Code:', placeholder='Enter school code')

Dropdown(description='Suggestions:', options=('Select',), value='Select')

Button(description='Search', icon='search', style=ButtonStyle(), tooltip='Click to search')

Button(button_style='warning', description='Clear', icon='times-circle', style=ButtonStyle(), tooltip='Click t…

Output()